In [1]:
pwd

'O:\\VINCI_NLPETL\\workspace\\jianlins\\PycharmProjects\\vectorsearch\\notebooks'

# Import

In [2]:
import os, sys

In [3]:
from pathlib import Path
import shutil
from loguru import logger
from bs4 import BeautifulSoup
from spacy.lang.en import English
from medspacy_io.reader.brat_reader import BratDocReader, BratDirReader
from spacy import displacy
import medspacy
import spacy
import pandas as pd
from spacy.tokens import SpanGroup
from medspacy_io.writer import EhostWriter
from medspacy_io.vectorizer import Vectorizer
import joblib
from spacy.tokens import Doc
from typing import List
pd.reset_option('display.width', None)

# Prepare brat annotation for NER training

In [4]:
train_dir=r'..\data\n2c2\training'
test_dir=r'..\data\n2c2\test'

In [5]:
cleaned_train_dir=r'..\data\n2c2\cleaned_training'
cleaned_test_dir=r'..\data\n2c2\cleaned_test'
Path(cleaned_train_dir).exists(), Path(cleaned_test_dir).exists()

(True, True)

In [6]:
# only keep entity annotations

In [7]:
def clean_annotations(source_dir):
    source_dir=Path(source_dir)
    target_dir=Path(source_dir.parent, 'cleaned_'+source_dir.name)
    if not target_dir.exists():
        target_dir.mkdir()
    for f in source_dir.glob('*.*'):
        if f.name.endswith('ann'):
            lines=[l for l in f.read_text().split('\n') if len(l.strip())>0  and l[0]=='T']
            content='\n'.join(lines)
        else:
            content=f.read_text()
        Path(target_dir, f.name).write_text(content)
    return target_dir

In [8]:
if not Path(cleaned_train_dir).exists():
   cleaned_rain_dir=clean_annotations(train_dir)

In [9]:
if not Path(cleaned_test_dir).exists():
    cleaned_test_dir=clean_annotations(test_dir)

## Load as spaCy docs

In [10]:
cleaned_train_dir=r'..\data\n2c2\cleaned_training'
cleaned_test_dir=r'..\data\n2c2\cleaned_test'
Path(cleaned_train_dir).exists(), Path(cleaned_test_dir).exists()

(True, True)

In [11]:
nlp=spacy.load('en_core_web_sm', disable=['ner'])

In [12]:
dir_reader = BratDirReader(nlp=nlp, schema_file=str(Path(cleaned_train_dir, 'annotation.conf')), support_overlap=True)

In [13]:
pickle_file= r'..\data\n2c2\spacy_docs.joblib'

In [14]:
if not Path(pickle_file).exists():
    train_docs=dir_reader.read(txt_dir=cleaned_train_dir)
    test_docs=dir_reader.read(txt_dir=cleaned_test_dir)
    print(len(train_docs), len(test_docs))
    joblib.dump((train_docs, test_docs), pickle_file)
else:
    print(f'{pickle_file} already exists, load them directly')
    # before load from pickle, initiate EhostDirReader or EhostDocReader first, because some Doc extension used to store meta data will not be automatically recreated by loading.
    train_docs, test_docs=joblib.load(pickle_file)

..\data\n2c2\spacy_docs.joblib already exists, load them directly


In [15]:
len(train_docs), len(test_docs)

(303, 202)

In [16]:
#train_docs

## Build embedding using sentence transformers

In [17]:
pickle_embedding_file= r'..\data\n2c2\embedding_df.joblib'

In [18]:
from sentence_transformers import SentenceTransformer

In [19]:
model=SentenceTransformer(r'D:\\pretrained_models\\models--sentence-transformers--all-mpnet-base-v2\\snapshots\\c46f31f8d119ac648208de9fba297c447a5a4474')

In [20]:
## Get all annotation types: 
annos=set()
for d in train_docs:
    for anno in d.spans.keys():
        annos.add(anno)
print(annos)

{'Dosage', 'Route', 'Frequency', 'Strength', 'Duration', 'ADE', 'Drug', 'Form', 'Reason'}


### to make embedding sentence ID consistent with train_df, we will all use conver2df from CRF wrapper

In [21]:
#def convert2df(docs: List[Doc], min_length:int=10):
#    data={'sentence':[], 'doc_name':[]}
#    for d in docs:
#        sents=[str(s) for s in d.sents if len(str(s).strip())>min_length]
#        data['sentence']+=sents
#        data['doc_name']+=[d._.doc_name]*len(sents)
#    return pd.DataFrame(data)
        

In [22]:
from CRFWrapper_Sentence import convert_docs, spans_to_bio


In [23]:
data, span2bioDF = spans_to_bio(train_docs[0], anno_types=annos)

In [24]:
span2bioDF

,sentence_id,doc_name,token,label,sentence
0,0,100035.txt,Admission,O,Admission Date:
1,0,100035.txt,Date,O,Admission Date:
2,0,100035.txt,:,O,Admission Date:
3,1,100035.txt,[,O,[**2115-2-22**] Discharge Date: ...
4,1,100035.txt,*,O,[**2115-2-22**] Discharge Date: ...
...,...,...,...,...,...
4491,263,100035.txt,Best,O,[**Location (un) **]\nCampus: EAST Best Pa...
4492,263,100035.txt,Parking,O,[**Location (un) **]\nCampus: EAST Best Pa...
4493,263,100035.txt,:,O,[**Location (un) **]\nCampus: EAST Best Pa...
4494,264,100035.txt,Main,O,Main Garage\n\n\n\n


In [25]:
span2bioDF.sentence[105]

'[**2078-8-9**]             Sex:   M\n\nService: MEDICINE\n\nAllergies:\nVicodin\n\nAttending:[**First Name3 (LF) 4891**]\nChief Complaint:\nPost-cardiac arrest, asthma exacerbation\n\nMajor Surgical or Invasive Procedure:\nIntubation\nRemoval of chest tubes placed at an outside hospital\nR CVL placement\n\n\nHistory of Present Illness:\nMr.'

In [26]:
#data

In [27]:
_, train_df=convert_docs(train_docs, anno_types=annos)
_, test_df=convert_docs(test_docs, anno_types=annos)
#train_df=convert2df(train_docs)

In [28]:
train_df

,sentence_id,doc_name,token,label,sentence
0,0,100035.txt,Admission,O,Admission Date:
1,0,100035.txt,Date,O,Admission Date:
2,0,100035.txt,:,O,Admission Date:
3,1,100035.txt,[,O,[**2115-2-22**] Discharge Date: ...
4,1,100035.txt,*,O,[**2115-2-22**] Discharge Date: ...
...,...,...,...,...,...
3151,929927,198406.txt,Followup,O,Mirtazapine 15 mg PO QHS\n\n\nDischarge Medica...
3152,929927,198406.txt,Instructions,O,Mirtazapine 15 mg PO QHS\n\n\nDischarge Medica...
3153,929927,198406.txt,:,O,Mirtazapine 15 mg PO QHS\n\n\nDischarge Medica...
3154,929927,198406.txt,\n,O,Mirtazapine 15 mg PO QHS\n\n\nDischarge Medica...


In [42]:
#train_df=train_df.rename_axis('sid').reset_index()
train_df['sentence'].to_list()
sentslength=[len(s) for s in d.sents ]
min(sentslength)
#for s in train_df['sentence'].to_list():
#    if len(s)<10:
#        print(s)

2

In [44]:
# get the unique sentence
sent_train_df = train_df[['sentence_id', 'sentence']]
uniqueSent_train_df = sent_train_df.drop_duplicates()
uniqueSent_train_df

,sentence_id,sentence
0,0,Admission Date:
3,1,[**2115-2-22**] Discharge Date: ...
35,2,[**2078-8-9**] Sex: M\n\nService...
113,3,[**Known lastname 3234**] is a 36 year old gen...
163,4,The patient initially presented to LGH ED with...
...,...,...
3059,929923,"Cyanocobalamin 1,000 mcg/mL Injection once a m..."
3071,929924,"Lorazepam 0.25 QAM, O.25 QPM, 0.5 mg QHS\n8 Ca..."
3086,929925,Cream Topical TID\n9.
3092,929926,Acetaminophen 1000 mg PO Q6H\n10.


In [39]:
#testEmbedding = model.encode(train_df['sentence'][3154].to_list()) #embedding input should be list

In [45]:
embeddings=model.encode(uniqueSent_train_df['sentence'].to_list())

In [46]:
embeddings.shape

(51798, 768)

In [47]:
len(list(embeddings))

51798

In [48]:
uniqueSent_train_df['embedding']=list(embeddings)

In [33]:
#train_df['embedding']=list(embeddings)

In [49]:
uniqueSent_train_df[:2]

,sentence_id,sentence,embedding
0,0,Admission Date:,"[0.026282, 0.03218903, -0.022386529, 0.0493732..."
3,1,[**2115-2-22**] Discharge Date: ...,"[0.016159855, 0.042264156, -0.018290585, -0.05..."


In [50]:
pickle_embedding_file= r'..\data\n2c2\embedding_df_uniqueSentID.joblib'
joblib.dump(uniqueSent_train_df, pickle_embedding_file)

['..\\data\\n2c2\\embedding_df_uniqueSentID.joblib']

## Build faiss index

In [64]:
faiss_index_path= r'..\data\n2c2\faiss_index_st768'

In [65]:
import faiss
base_index=faiss.IndexFlatL2(768)
index=faiss.IndexIDMap(base_index)


In [66]:
import numpy as np

In [67]:
%%time
index.add_with_ids(np.stack(uniqueSent_train_df['embedding']).astype(np.float32), uniqueSent_train_df['sentence_id'].tolist())
faiss.write_index(index, faiss_index_path)

CPU times: total: 688 ms
Wall time: 1.12 s


In [68]:
#%%time
#index.add_with_ids(np.stack(train_df['embedding']).astype(np.float32), train_df['sid'].tolist())
#faiss.write_index(index, faiss_index_path)

In [69]:
D, I=index.search(np.stack([uniqueSent_train_df.iloc[8].embedding]), 800000)

In [70]:
p=[v for v in D[0] if v>-1 and v<2.4028235e+38]

In [71]:
len(p)

51798

In [72]:
p[-10:]

[2.144607,
 2.146985,
 2.1478677,
 2.1491637,
 2.1497927,
 2.1672182,
 2.1730304,
 2.1730824,
 2.1897774,
 2.2237568]

In [73]:
uniqueSent_train_df.shape

(51798, 3)

In [74]:
uniqueSent_train_df#.iloc[I[0]]#[['sentence', 'sentence_id']]

,sentence_id,sentence,embedding
0,0,Admission Date:,"[0.026282, 0.03218903, -0.022386529, 0.0493732..."
3,1,[**2115-2-22**] Discharge Date: ...,"[0.016159855, 0.042264156, -0.018290585, -0.05..."
35,2,[**2078-8-9**] Sex: M\n\nService...,"[0.025958579, -0.05749655, 0.012378361, -0.009..."
113,3,[**Known lastname 3234**] is a 36 year old gen...,"[0.023170307, 0.03989108, 0.026217388, -0.0272..."
163,4,The patient initially presented to LGH ED with...,"[0.008176211, -0.06342948, 0.048615105, -0.045..."
...,...,...,...
3059,929923,"Cyanocobalamin 1,000 mcg/mL Injection once a m...","[0.050521564, -0.08905716, -0.0019493615, -0.0..."
3071,929924,"Lorazepam 0.25 QAM, O.25 QPM, 0.5 mg QHS\n8 Ca...","[-0.030010266, -0.062390286, 0.00167252, 0.016..."
3086,929925,Cream Topical TID\n9.,"[0.026732022, -0.04987913, 0.024520764, -0.016..."
3092,929926,Acetaminophen 1000 mg PO Q6H\n10.,"[-0.017295217, -0.10513715, -0.0030776137, -0...."


In [76]:
I[0]

array([     8,    167, 699572, ...,     -1,     -1,     -1], dtype=int64)

In [48]:
train_df

,sid,sentence,doc_name,embedding
0,0,Admission Date:,100035.txt,"[0.026282, 0.03218903, -0.022386529, 0.0493732..."
1,1,[**2115-2-22**] Discharge Date: ...,100035.txt,"[0.016159855, 0.042264156, -0.018290585, -0.05..."
2,2,[**2078-8-9**] Sex: M\n\nService...,100035.txt,"[0.025958579, -0.05749655, 0.012378361, -0.009..."
3,3,[**Known lastname 3234**] is a 36 year old gen...,100035.txt,"[0.023170307, 0.03989108, 0.026217388, -0.0272..."
4,4,The patient initially presented to LGH ED with...,100035.txt,"[0.008176211, -0.06342948, 0.048615105, -0.045..."
...,...,...,...,...
47525,47525,"Cyanocobalamin 1,000 mcg/mL Injection once a m...",198406.txt,"[0.050521564, -0.08905716, -0.0019493615, -0.0..."
47526,47526,"Lorazepam 0.25 QAM, O.25 QPM, 0.5 mg QHS\n8 Ca...",198406.txt,"[-0.030010268, -0.062390275, 0.0016725484, 0.0..."
47527,47527,Cream Topical TID\n9.,198406.txt,"[0.026732022, -0.04987913, 0.024520764, -0.016..."
47528,47528,Acetaminophen 1000 mg PO Q6H\n10.,198406.txt,"[-0.017295217, -0.10513715, -0.0030776137, -0...."


In [77]:
sdf=train_df[:500]

In [78]:
sdf.shape

(500, 5)

In [79]:
train_df

,sentence_id,doc_name,token,label,sentence
0,0,100035.txt,Admission,O,Admission Date:
1,0,100035.txt,Date,O,Admission Date:
2,0,100035.txt,:,O,Admission Date:
3,1,100035.txt,[,O,[**2115-2-22**] Discharge Date: ...
4,1,100035.txt,*,O,[**2115-2-22**] Discharge Date: ...
...,...,...,...,...,...
3151,929927,198406.txt,Followup,O,Mirtazapine 15 mg PO QHS\n\n\nDischarge Medica...
3152,929927,198406.txt,Instructions,O,Mirtazapine 15 mg PO QHS\n\n\nDischarge Medica...
3153,929927,198406.txt,:,O,Mirtazapine 15 mg PO QHS\n\n\nDischarge Medica...
3154,929927,198406.txt,\n,O,Mirtazapine 15 mg PO QHS\n\n\nDischarge Medica...


In [81]:
#sdf['y'].unique()

In [132]:
gsdf=list(sdf.groupby('y'))

In [139]:
[(k,v.shape[0]) for k,v in gsdf]

[('ADE', 2),
 ('Dosage', 26),
 ('Drug', 87),
 ('Duration', 14),
 ('Form', 44),
 ('Frequency', 29),
 ('NEG', 208),
 ('Reason', 29),
 ('Route', 32),
 ('Strength', 29)]